Task: Data extraction through API (data source: Electrica De Espana)
Source: https://www.ree.es/es/apidatos

In [2]:
import pandas as pd
import json
import requests
import matplotlib.pyplot as plt

In [3]:
def co2_evolution(start_date, end_date):
    #Enter the start_date and end_date in the following format: YYYY-MM-DDTHH:MM

    # API get request
    response = requests.get(f"https://apidatos.ree.es/en/datos/generacion/evolucion-estructura-generacion-emisiones-asociadas?start_date={start_date}&end_date={end_date}&time_trunc=day")
    response_json = response.json()
    data = response_json
    
    # Accessing data for "non-renewable type (Con emisiones de CO2 eq.)" ("id": 1)
    data1 = data["included"][0]["attributes"]["values"]
    evolution_co2_1 = pd.DataFrame(data1)
    evolution_co2_1["Type_id"] = 1
    evolution_co2_1["Description"] = "CO2 emission eq."

    # Accessing data for "renewable type (Sin emisiones de CO2 eq.)" ("id": 2)
    data2 = data["included"][1]["attributes"]["values"]
    evolution_co2_2 = pd.DataFrame(data2)
    evolution_co2_2["Type_id"] = 2
    evolution_co2_2["Description"] = "No CO2 emission eq."

    # Concat dataframes
    frames = [evolution_co2_1, evolution_co2_2]
    evolution_co2 = pd.concat(frames).reset_index(drop=True)
    evolution_co2['datetime'] = pd.to_datetime(evolution_co2["datetime"],format='%Y-%m')
    evolution_co2 = evolution_co2.rename(columns={'value': 'Value', "datetime": "Date", "percentage": "Percentage"})
    return evolution_co2

In [4]:
co2_evolution("2020-01-01T00:00", "2020-06-15T00:00")

,Value,Percentage,Date,Type_id,Description
0,202459.6665,0.340023,2020-01-01 00:00:00+01:00,1,CO2 emission eq.
1,251804.3225,0.355718,2020-01-02 00:00:00+01:00,1,CO2 emission eq.
2,236659.0320,0.345087,2020-01-03 00:00:00+01:00,1,CO2 emission eq.
3,194786.5415,0.289596,2020-01-04 00:00:00+01:00,1,CO2 emission eq.
4,199303.8105,0.325875,2020-01-05 00:00:00+01:00,1,CO2 emission eq.
...,...,...,...,...,...
329,476097.9560,0.708120,2020-06-11 00:00:00+02:00,2,No CO2 emission eq.
330,531540.9020,0.768363,2020-06-12 00:00:00+02:00,2,No CO2 emission eq.
331,454318.6560,0.720553,2020-06-13 00:00:00+02:00,2,No CO2 emission eq.
332,365710.4410,0.651768,2020-06-14 00:00:00+02:00,2,No CO2 emission eq.


In [1]:
# Get the data for 2020 and 2019
co2_20 = co2_evolution("2020-01-01T00:00", "2020-06-15T00:00")
co2_19 = co2_evolution("2019-01-01T00:00", "2019-06-15T00:00")

NameError: name 'co2_evolution' is not defined

In [ ]:
# Export the data in a .csv format
co2_20 = co2_20.to_csv("clean_db/CO2-Evolution_2020.csv", index=False)
co2_19 = co2_19.to_csv("clean_db/CO2-Evolution_2019.csv", index=False)

In [ ]:
# Analising the data
co2_evolution("2020-01-01T00:00", "2020-06-15T00:00").plot();

## Alternatively

In [ ]:
response = requests.get(f"https://apidatos.ree.es/en/datos/generacion/evolucion-estructura-generacion-emisiones-asociadas?start_date=2020-01-01T00:00&end_date=2020-06-15T00:00&time_trunc=day")
response_json = response.json()
data = response_json
data = data["included"]

In [ ]:
df = pd.DataFrame(columns=['Value', "Percentage", 'Date', 'Type_id', 'Description'])

for t in data:
    desc = t["type"]
    values_list = t["attributes"]["values"]
    df_values = (pd.DataFrame(values_list)
                 .assign(Description=desc)
                 .rename(columns={"value": "Value",
                                  "percentage": "Percentage",
                                  "datetime": "Date"}))
    df = pd.concat([df, df_values])

In [ ]:
df